In [1]:
import sklearn
import random 

# Pyspark Library #
# SQL
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit
# ML
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import os
import pandas as pd
import numpy as np

In [2]:
#Make Spark Session
spark = SparkSession.builder.config( "spark.jars","/root/hadoop/share/hadoop/tools/lib/mysql-connector-java-8.0.27.jar") \
    .master("local").appName("ml_reco").getOrCreate()

In [6]:
#Load tables (buylist, product, category_small, user_recommand)
buylist = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "buylist") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
product = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "product") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
category_small = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "category_small") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
user_recommand = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "user_recommand") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
cart = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "cart") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()

In [5]:
buylist.toPandas()

,id,user_id,product_id,count,buy_date
0,1,1,1,"[49, 0, 0, 0, 0]",2022-02-09
1,2,1,2,"[50, 0, 0, 0, 0]",2022-01-12
2,3,1,3,"[51, 0, 0, 0, 0]",2022-01-21
3,4,1,4,"[49, 0, 0, 0, 0]",2021-08-19
4,5,1,5,"[49, 0, 0, 0, 0]",2021-03-24


In [3]:
product.toPandas()

,id,category_small_id,name,price,image,brand,avg_star,regist_time
0,1,1,캐럿 남성 래귤러핏 터틀넥 티,11900,,캐럿,4.5,22.02.08
1,2,1,나이키 반팔,10000,,나이키,3.3,22.01.09
2,3,1,나이키 긴팔,10000,,나이키,2.8,22.01.09
3,4,3,아디다스 운동화,110000,,아디다스,4.3,21.06.13
4,5,3,아식스 운동화,90000,,아식스,4.2,20.12.23


In [4]:
category_small.toPandas()

,id,category_mid_id,name
0,1,1,의류
1,2,1,속옷/잠옷
2,3,1,신발
3,4,1,가방/잡화


In [7]:
cart.toPandas()

,user_id,product_id


In [ ]:
data = c